Scraping 2

In [6]:
import pandas as pd
import requests
import io
import re
import time

# 1. CONFIGURATION ET REQUÊTE (Bonnes pratiques du cours)
url = "https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France"
headers = {
    "User-Agent": "Etudiant_DataScience_Scraping_Exercice/1.0"
}

response = requests.get(url, headers=headers)

dfs = pd.read_html(io.StringIO(response.text), match="Insee")
df_pref = dfs[0].copy()

# 2. NETTOYAGE DES DONNÉES

def clean_text(text):
    if isinstance(text, str):
        # Enlève les références [1], [a]...
        text = re.sub(r'\[.*?\]', '', text)
        # Enlève les précisions entre parenthèses comme (capitale) ou (Chef-lieu)
        # sauf si c'est vital, mais ici "Paris (capitale)" gêne le géocodage
        text = re.sub(r'\(.*?\)', '', text)
        return text.strip()
    return text


print(df_pref.head())

  No Insee              Département       Préfecture  \
0      010                      Ain  Bourg-en-Bresse   
1      020                    Aisne             Laon   
2      030                   Allier          Moulins   
3      040  Alpes-de-Haute-Provence  Digne-les-Bains   
4      050             Hautes-Alpes              Gap   

  Population communale[1] (dernière pop. légale)  \
0                                         42 065   
1                                         24 066   
2                                         19 344   
3                                         17 694   
4                                         40 656   

                                    Sous-préfectures  
0                                Belley, Gex, Nantua  
1  Château-Thierry, Saint-Quentin, Soissons, Vervins  
2                                   Montluçon, Vichy  
3             Barcelonnette, Castellane, Forcalquier  
4                                           Briançon  
